# Creating ASN File for splittag Products

In [2]:
from costools import timefilter

import calcos 

from astropy.io import fits                                            
import numpy as np
import glob
import os

The following tasks in the costools package can be run with TEAL:
         splittag                 timefilter                 x1dcorr


### timefilter example from https://costools.readthedocs.io/en/latest/timefilter.html

In [6]:
timefilter.TimelineFilter("lb3q01meq_corrtag_a.fits", "temp_corrtag_a.fits",  # your input names will be different
                              "sun_alt > 0.")
timefilter.TimelineFilter("lb3q01meq_corrtag_b.fits", "temp_corrtag_b.fits",
                              "sun_alt > 0.")
# repeat this for all exposures

### Creating an ASN file out of the above files

In [10]:
# this properly assigns the type of exposure in a way that CalCOS will recognize
type_dict = {'WAVECAL' : 'EXP-AWAVE',
             'EXTERNAL/SCI' : 'EXP-FP'}

files = glob.glob('temp_corrtag_a*') ## This will be different depending on your output name ## only need the A segment in the ASN, and calcos will find B for you

for f in files:
    
    # Adding the file details to the association table
    rootnames = [f]  # MEMNAME
    types = [type_dict[fits.getval(f, 'EXPTYPE')]] # MEMTYPE
    included = [True] # MEMPRSNT
    
    # Adding the ASN details to the end of the association table
    # the rootname needs to be the full name, not just the rootname
    asn_root = os.path.basename(f.split('corrtag')[0][:-1]) # removing the extra "_" at the end of this
    rootnames.append(asn_root.upper()) # MEMNAME
    types.append('PROD-FP') # MEMTYPE
    included.append(True) # MEMPRSNT
    
    # Putting together the fits table
    #   40 is the number of characters allowed in this field. If your rootname is longer than 40, 
    #     you will need to increase this
    c1 = fits.Column(name='MEMNAME', array=np.array(rootnames), format='40A') 
    c2 = fits.Column(name='MEMTYPE', array=np.array(types), format='14A')
    c3 = fits.Column(name='MEMPRSNT', format='L', array=included)
    t = fits.BinTableHDU.from_columns([c1, c2, c3])
    
    # Writing the fits table
    t.writeto(asn_root.lower()+'_asn.fits')

    print('Saved: {}'.format(asn_root.lower()+'_asn.fits'))


Saved: temp_asn.fits


In [11]:
asnfile = 'temp_asn.fits'
print(fits.getdata(asnfile))

[('temp_corrtag_a.fits', 'EXP-FP',  1) ('TEMP', 'PROD-FP',  1)]


### Then run CalCOS on those individual files

In [12]:
for f in glob.glob('temp_asn.fits'):
    calcos.calcos(f, outdir='calibrated_filtered')

CALCOS version 3.3.7 (2019-07-09)
numpy version 1.16.4
astropy version 3.2.1
Begin 10-Apr-2020 10:50:18 EDT
Association file = temp_asn.fits

TIME-TAG calibration -- 10-Apr-2020 10:50:20 EDT
Input     temp_corrtag_a.fits
OutTag    calibrated_filtered/temp_corrtag_a.fits
OutFlt    calibrated_filtered/temp_flt_a.fits
OutCounts calibrated_filtered/temp_counts_a.fits
OutFlash  calibrated_filtered/temp_lampflash_a.fits
DETECTOR  FUV, segment A
EXPTYPE   EXTERNAL/SCI
OPT_ELEM  G160M, CENWAVE 1623, FPOFFSET 0
APERTURE  PSA

BADTCORR  OMIT
RANDCORR  OMIT (already complete)
TEMPCORR  OMIT (already complete)
GEOCORR   OMIT (already complete)
DGEOCORR  OMIT
DEADCORR  OMIT (already complete)
PHACORR   OMIT (already complete)
DOPPCORR  OMIT (already complete)
XTRACTAB= lref$x1v17414l_1dx.fits
DISPTAB = lref$05i1639ml_disp.fits
BRFTAB  = lref$x1u1459il_brf.fits
FLATCORR  OMIT (already complete)
WAVECORR  OMIT (already complete)
BRSTCORR  OMIT
TRCECORR  OMIT
ALGNCORR  OMIT
DQICORR   PERFORM (complete

FUVA spectrum was found at y = 480.11 vs. nominal y = 478.06
    error estimate for y location = 8.52, FWHM = 4.06
Spectrum will be extracted at y = 478.06

TIME-TAG calibration -- 10-Apr-2020 10:50:41 EDT
Input     temp_corrtag_b.fits
OutTag    calibrated_filtered/temp_corrtag_b.fits
OutFlt    calibrated_filtered/temp_flt_b.fits
OutCounts calibrated_filtered/temp_counts_b.fits
OutFlash  calibrated_filtered/temp_lampflash_b.fits
DETECTOR  FUV, segment B
EXPTYPE   EXTERNAL/SCI
OPT_ELEM  G160M, CENWAVE 1623, FPOFFSET 0
APERTURE  PSA

BADTCORR  OMIT
RANDCORR  OMIT (already complete)
TEMPCORR  OMIT (already complete)
GEOCORR   OMIT (already complete)
DGEOCORR  OMIT
DEADCORR  OMIT (already complete)
PHACORR   OMIT (already complete)
DOPPCORR  OMIT (already complete)
XTRACTAB= lref$x1v17414l_1dx.fits
DISPTAB = lref$05i1639ml_disp.fits
BRFTAB  = lref$x1u1459il_brf.fits
FLATCORR  OMIT (already complete)
WAVECORR  OMIT (already complete)
BRSTCORR  OMIT
TRCECORR  OMIT
ALGNCORR  OMIT
DQICORR   PE